## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [153]:
DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0180.shtml"

Устанавливаем библиотеки

In [154]:
import warnings
warnings.filterwarnings('ignore')
!pip -q install rnnmorph

Создаём объект морфологического анализатора `RNNMorph`

In [155]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [156]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текст с html тегами

In [157]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/t/tolstoj_a_k/>Толстой Алексей Константинович</a><br>\r\n'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [158]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [159]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака\n\n\n\nТолстой Алексей Константинович\r\nСемья вурдалака\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n['

In [160]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /Users/muzafarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 576 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [161]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться



9/9 [==============================] - 2s 166ms/step


sentences: 100%|██████████| 576/576 [00:00<00:00, 637109.47it/s]


[['другой',
  'ребёнок',
  'я',
  'такой',
  'же',
  'образ',
  'кинуть',
  'вслед',
  ',',
  'но',
  'он',
  'упасть',
  'прямо',
  'под',
  'копыто',
  'лошадь',
  'и',
  'быть',
  'раздавленный',
  '.']]

In [162]:
import re

def clean_string(string):
    # Удаляем английские слова
    string = re.sub(r'\b[a-zA-Z]+\b', '', string)
    # Удаляем все символы кроме русских слов, цифр и некоторых специальных символов
    return re.sub(r'[0-9a-z.,!?:;"\'()\[\]#*@$&--^+-/{}|<>]', '', string)

!@#$^&%*()+=-[]\/{}|:<>?,.
# Function to process nested lists and remove empty elements
def process_list(lst):
    i = 0
    while i < len(lst):
        if isinstance(lst[i], list):
            process_list(lst[i])
            if not lst[i]:
                lst.pop(i)
                continue
        elif isinstance(lst[i], str):
            cleaned_item = clean_string(lst[i])
            if cleaned_item:
                lst[i] = cleaned_item
            else:
                lst.pop(i)
                continue
        i += 1


# Clean the original list
process_list(predictions)

print(predictions)

zsh:1: no matches found: %*
zsh:1: no matches found: ?,.


[['классика', 'толстый', 'алексей', 'константинович'], ['семья', 'вурдалак', 'толстый', 'алексей', 'константинович', 'семья', 'вурдалак', 'классик', 'регистрация', 'найти', 'рейтинг', 'обсуждение', 'новинка', 'обзор', 'помощь', 'комментарий', 'последний', 'от'], ['толстый', 'алексей', 'константинович', 'год', 'обновлённый'], ['статистика'], ['глава', 'проза', 'фантастика', 'проза', 'скачать', 'оценка', 'ваш', 'оценка', 'шедевр', 'замечательный', 'очень', 'хорошо', 'хорошо', 'нормально', 'не', 'читать', 'терпимо', 'посредственно', 'плохо', 'очень', 'плохо', 'не', 'читать', 'алексей', 'константинович', 'толстой'], ['семья', 'вурдалак', 'оригинал', 'здесь', 'библиотека', 'альдебаран', 'неизданный', 'отрывок', 'из', 'записка', 'неизвестный', 'примечание', 'оригинал', 'написанный', 'понемецки'], ['в', 'год', 'в', 'вена', 'собраться', 'цвет', 'европейский', 'образованность', 'дипломатический', 'дарование', 'весь', 'тот', 'что', 'блистать', 'в', 'тогдашний', 'общество'], ['но', 'вот', 'конгре

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [163]:
len(predictions)

574

In [164]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 


8683

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [166]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/muzafarov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что']

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
